In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, BaggingRegressor
from sklearn.naive_bayes import GaussianNB

In [2]:
!pip install xgboost

In [7]:
!pip install catboost

In [8]:
import xgboost as xgb
from catboost import CatBoostClassifier

In [9]:
#как в первой лабораторной, преобразуем датасет
df = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv', sep=',')
# print(df.info())

df = pd.concat([pd.get_dummies(df['Gender'], prefix='Gender', dtype='float64'), df], axis=1)
df.drop("Gender", axis=1, inplace=True)
# print(df.info())

# print(df['family_history_with_overweight'].value_counts())
df = pd.concat([pd.get_dummies(df['family_history_with_overweight'], prefix='fam_his_ow', dtype='float64'), df], axis=1)
df.drop("family_history_with_overweight", axis=1, inplace=True)
# print(df.info())
# print(df['FAVC'].value_counts())

df = pd.concat([pd.get_dummies(df['FAVC'], prefix='FAVC', dtype='float64'), df], axis=1)
df.drop("FAVC", axis=1, inplace=True)
# print(df.info())

# print(df['CAEC'].value_counts())
df["CAEC"] = df["CAEC"].map({"Sometimes": 1.0, "Frequently": 2, "Always": 3, "no": 0})
# print(df['CAEC'].value_counts())
# print(df.info())


# print(df['SMOKE'].value_counts())
df = pd.concat([pd.get_dummies(df['SMOKE'], prefix='SMOKE', dtype='float64'), df], axis=1)
df.drop("SMOKE", axis=1, inplace=True)
# print(df.info())

# print(df['SCC'].value_counts())
df = pd.concat([pd.get_dummies(df['SCC'], prefix='SCC', dtype='float64'), df], axis=1)
df.drop("SCC", axis=1, inplace=True)
# print(df.info())

# print(df['CALC'].value_counts())
df["CALC"] = df["CALC"].map({"Sometimes": 1.0, "Frequently": 2, "Always": 3, "no": 0})
# print(df.info())

# print(df['MTRANS'].value_counts())
df["MTRANS"] = df["MTRANS"].map(
    {"Public_Transportation": 0.0, "Automobile": 1, "Walking": 2, "Motorbike": 3, "Bike": 4})
# print(df.info())

# print(df['NObeyesdad'].value_counts())
df["NObeyesdad"] = df["NObeyesdad"].map(
    {"Obesity_Type_I": 4,  # тип ожирения
     "Obesity_Type_II": 5,
     "Obesity_Type_III": 6,
     "Overweight_Level_I": 2,  # Уровень избыточного веса
     "Overweight_Level_II": 3,
     "Normal_Weight": 1,
     "Insufficient_Weight": 0.0})  # недостаточный вес

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SCC_no          2111 non-null   float64
 1   SCC_yes         2111 non-null   float64
 2   SMOKE_no        2111 non-null   float64
 3   SMOKE_yes       2111 non-null   float64
 4   FAVC_no         2111 non-null   float64
 5   FAVC_yes        2111 non-null   float64
 6   fam_his_ow_no   2111 non-null   float64
 7   fam_his_ow_yes  2111 non-null   float64
 8   Gender_Female   2111 non-null   float64
 9   Gender_Male     2111 non-null   float64
 10  Age             2111 non-null   float64
 11  Height          2111 non-null   float64
 12  Weight          2111 non-null   float64
 13  FCVC            2111 non-null   float64
 14  NCP             2111 non-null   float64
 15  CAEC            2111 non-null   float64
 16  CH2O            2111 non-null   float64
 17  FAF             2111 non-null   f

In [10]:
y = df["Weight"]  # вытаскивает(удаляет из таблицы) - мы должны спрогнозировать это значение
df.drop("Weight", axis=1, inplace=True)
x = df

In [24]:
#по какой то причине, несмотря на то, что у меня везде тип float64, при обучении почему то y считается как object, поэтому я преобразовала его в int
y=y.astype('int') 
y

0        64
1        56
2        77
3        87
4        89
       ... 
2106    131
2107    133
2108    133
2109    133
2110    133
Name: Weight, Length: 2111, dtype: int32

In [55]:
x_train, x_test, y_train, y_test=train_test_split(x,y)
dtc=DecisionTreeClassifier()

dtc.fit(x,y)
#dtc.fit(x_train, y_train)
#dtc.fit(x_test, y_test)
print(dtc.score(x_train,y_train)) # 1.0
print(dtc.score(x_test,y_test)) # 1.0
print(dtc.score(x,y)) #1.0

1.0
1.0
1.0


In [57]:
lr=LogisticRegression(max_iter=1000) # я брала max_iter =1000, 1500, 2000, 3000 результат особо не менялся, при большем кол-ве итераций уже долго считается 

lr.fit(x, y) 
#dtc.fit(x_train, y_train)
#dtc.fit(x_test, y_test)
print(dtc.score(x_train,y_train)) # 1.0
print(dtc.score(x_test,y_test)) # 1.0
print(dtc.score(x,y)) #1.0

1.0
1.0
1.0


D:\anac\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [65]:
rfc = RandomForestClassifier(max_depth=10, n_estimators=200, max_samples=None) 
# ------------------------train--------------------test---------------------x,y----------
# max_depth=5        0.43714466203411245        0.6363636363636364          0.4109848484848485
# max_depth=10       0.9576753000631711         0.9981060606060606          0.9393939393939394

In [66]:
#rfc.fit(x_train, y_train)
#rfc.fit(x_test, y_test)
rfc.fit(x,y)
print(rfc.score(x_train, y_train))
print(rfc.score(x, y))
print(rfc.score(x_test, y_test))


0.950726468730259
0.9478919943154903
0.9393939393939394


In [48]:
cat = CatBoostClassifier(n_estimators=300, max_depth=, learning_rate=1.5)
cat.fit(x_train, y_train)
print('Catboost on train set: ', cat.score(x_train, y_train))

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


0:	learn: 4.6382948	total: 420ms	remaining: 2m 5s
1:	learn: 3.4944694	total: 581ms	remaining: 1m 26s
2:	learn: 3.1258093	total: 723ms	remaining: 1m 11s
3:	learn: 2.8452599	total: 860ms	remaining: 1m 3s
4:	learn: 2.3334395	total: 1s	remaining: 59.2s
5:	learn: 2.0973788	total: 1.13s	remaining: 55.2s
6:	learn: 1.8675624	total: 1.24s	remaining: 52s
7:	learn: 1.7138613	total: 1.35s	remaining: 49.3s
8:	learn: 1.5986500	total: 1.5s	remaining: 48.6s
9:	learn: 1.4869481	total: 1.64s	remaining: 47.5s
10:	learn: 1.3875154	total: 1.76s	remaining: 46.1s
11:	learn: 1.3132925	total: 1.86s	remaining: 44.7s
12:	learn: 1.2164724	total: 1.98s	remaining: 43.6s
13:	learn: 1.1427977	total: 2.08s	remaining: 42.6s
14:	learn: 1.0959109	total: 2.19s	remaining: 41.6s
15:	learn: 1.0385913	total: 2.31s	remaining: 40.9s
16:	learn: 0.9882726	total: 2.41s	remaining: 40.1s
17:	learn: 0.9456316	total: 2.54s	remaining: 39.8s
18:	learn: 0.9108615	total: 2.67s	remaining: 39.5s
19:	learn: 0.8607477	total: 2.78s	remaining: 

162:	learn: 0.0697508	total: 19.2s	remaining: 16.1s
163:	learn: 0.0692763	total: 19.3s	remaining: 16s
164:	learn: 0.0686974	total: 19.4s	remaining: 15.9s
165:	learn: 0.0681624	total: 19.5s	remaining: 15.8s
166:	learn: 0.0674424	total: 19.6s	remaining: 15.6s
167:	learn: 0.0667792	total: 19.7s	remaining: 15.5s
168:	learn: 0.0662235	total: 19.8s	remaining: 15.4s
169:	learn: 0.0655597	total: 20s	remaining: 15.3s
170:	learn: 0.0652242	total: 20.1s	remaining: 15.1s
171:	learn: 0.0648003	total: 20.2s	remaining: 15s
172:	learn: 0.0642447	total: 20.3s	remaining: 14.9s
173:	learn: 0.0637539	total: 20.4s	remaining: 14.8s
174:	learn: 0.0633183	total: 20.5s	remaining: 14.7s
175:	learn: 0.0628611	total: 20.7s	remaining: 14.6s
176:	learn: 0.0622108	total: 20.8s	remaining: 14.5s
177:	learn: 0.0618719	total: 20.9s	remaining: 14.3s
178:	learn: 0.0616179	total: 21s	remaining: 14.2s
179:	learn: 0.0610822	total: 21.1s	remaining: 14.1s
180:	learn: 0.0604073	total: 21.3s	remaining: 14s
181:	learn: 0.0598967	

In [82]:
vote = VotingClassifier(estimators=[('logreg', lr), ('dtc', dtc)], voting='soft')
vote.fit(x_train, y_train)
print('Vote on train set: ', vote.score(x_train, y_train))
print('Vote on test set: ', vote.score(x_test, y_test))

Vote on train set:  1.0
Vote on test set:  0.3806818181818182


D:\anac\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [69]:
bag = BaggingClassifier(dtc, n_estimators=100, max_features=0.6, max_samples=0.5)
#bag.fit(x_train, y_train)
#bag.fit(x_test, y_test)
bag.fit(x, y)

print( bag.score(x_train, y_train))
print( bag.score(x_test, y_test))
print( bag.score(x, y))
#тут улучшать нечего, итак хороший результат 0.99...

0.9936828806064435
0.9962121212121212
0.9943154902889626


In [72]:
r=RandomForestRegressor(n_estimators=100, max_depth=10)

#r.fit(x_train, y_train)
#r.fit(x_test, y_test)
r.fit(x, y)

print( r.score(x_train, y_train))
print( r.score(x_test, y_test))
print( r.score(x, y))
# n_estimators=100, max_depth=5    0.9834830806881444       0.9774426207693254            0.9820108836053946
#при n_estimators=10, max_depth=10 результат 0.99
# при n_estimators=100, max_depth=10 тоже примерно 0.99

0.9971491567446016
0.9965906561935879
0.9970133807624019


In [79]:
b=BaggingRegressor(n_estimators=100)

#b.fit(x_train, y_train)
#b.fit(x_test, y_test)
b.fit(x, y)

print( b.score(x_train, y_train))
print( b.score(x_test, y_test))
print( b.score(x, y))
#n_estimators 10, 50, 100 результат всегда примерно 0.99

0.9984232708978364
0.9977538390032111
0.9982600493570091
